In [1]:
import pandas as pd
import os
import time
import sys
sys.path.append(os.path.dirname(os.getcwd()))
from model import LineageMantaObject
from utils import create_raw_table, transfrom_column_name_before_create_table, convert_df_to_oracle_format
from config import NODE_PATH, LINEAGE_SOURCE, LINEAGE_TARGET,EXPORT_MANTA_FILE_PATH, LINEAGE_MANTA_TABLE_NAME_RAW, TABLE_MANTA_LINEAGE_RAW_COLUMN, LINEAGE_MANTA_TABLE_FLOW_TEST, TABLE_MANTA_LINEAGE_FLOW
from repo import RepoLineage

os.getcwd(), c:\Users\hungpv6\OneDrive - hdbank.com.vn\Manage jobs\Lineage_manta\main


In [2]:

if __name__ == '__main__':
    start_time = time.time()
    object = LineageMantaObject(lineage_name= 'manta')
    df_raw, table_name_list = create_raw_table(object)
    df_graph = None
    pd.set_option('display.max_rows', None)
    for table_name in table_name_list:
        df = df_raw.query(f"Table_name == '{table_name}' ")
        # df = df_raw.loc[df_raw['TABLE_NAME'] == table_name]
        df_describe = object.describe_table(
            df_path=df,
            source_col=LINEAGE_SOURCE,
            target_col=LINEAGE_TARGET,

        )
        all_values = list(df_describe['all_values'])
        redact_node_df = object.redact_node_with_int(total_nodes_df=df)
        combo_list = redact_node_df[NODE_PATH].tolist()
        dictionary = object.mapping_dict_nodepath(df =redact_node_df)
        adjacency_list = object.create_adjacency_list(dictionary = dictionary )
        combo_list_copy = combo_list.copy()
        
        graph = object.deploy_lineage_manta(df_raw= df, 
                                            combo_list_copy=combo_list_copy, 
                                            dictionary=dictionary,
                                            adjacency_list = adjacency_list, 
                                            source_col=LINEAGE_SOURCE, 
                                            target_col=LINEAGE_TARGET)
        object.add_name_of_table_column(df = graph, table_name = table_name)
        if df_graph is None:
            df_graph = graph
        else:
            df_graph = pd.concat([df_graph, graph])
        end_time = time.time()
        if (int(end_time) - int(start_time)) >= 300:
            print(f"Time elapsed --- need impove the algorithm: {end_time - start_time} seconds")
            break
    df_graph = object.extract_components(table = df_graph, column_extract='Raw_Node')
    df_graph = transfrom_column_name_before_create_table(df=df_graph)


LINEAGE_MANTA_W4
TABLE_NAME =  FM_CLIENT_CONTACT | FILE_NAME=  relations_FM_CLIENT_CONTACT | file_path =  C:/Users/hungpv6/Downloads/manta_data_file/linage_manta_export_file/relations_FM_CLIENT_CONTACT.csv


KeyboardInterrupt: 

In [3]:
# print(type(df_graph))
# print(df_graph.head(10))


In [4]:
# print(df_graph.head(10)) 

In [5]:
database = 'test'
enviroment = 'DEV'
db_connection = RepoLineage()
db_connect = db_connection.get_connection(db_name='test',environment = 'DEV', user = 'dtc')
# data = db_connection.create_lineage_manta_table(connection = db_connect,
#                                                 table_name = LINEAGE_MANTA_TABLE_NAME_RAW,
#                                                 column_definitions= TABLE_MANTA_LINEAGE_RAW_COLUMN)
# db_connection.insert_data_from_dataframe(connection = db_connect, table_name = LINEAGE_MANTA_TABLE_NAME_RAW, dataframe=convert_df_to_oracle_format(transfrom_column_name_before_create_table(df_raw)))

Connection to database 'test' established successfully.


In [12]:
data = db_connection.create_lineage_manta_table(connection = db_connect,
                                                table_name = LINEAGE_MANTA_TABLE_FLOW_TEST,
                                                column_definitions= TABLE_MANTA_LINEAGE_FLOW)



In [13]:
db_connection.insert_data_from_dataframe(connection = db_connect, table_name = LINEAGE_MANTA_TABLE_FLOW_TEST, dataframe=convert_df_to_oracle_format(transfrom_column_name_before_create_table(df_graph)))

In [8]:
# data = db_connection.get_data(connection = db_connect, query = f"SELECT * FROM {LINEAGE_MANTA_TABLE_NAME_RAW} WHERE ROWNUM <= 1000")
# print(data)

In [ ]:
data = db_connection.get_data(connection = db_connect, query = f"SELECT * FROM {LINEAGE_MANTA_TABLE_FLOW_TEST} WHERE ROWNUM <= 10")
print(data)

[('ACCOUNT', 'Oracle.W4DBN4.HDB.YTACCOUNT.YTACCOUNT_BODY.<8,1>PLSQL_BLOCK.<12,14>YTACCOUNT.REC.REMOTE_VERSION,Oracle.W4DBN4.HDB.YGACCOUNT.REC.REMOTE_VERSION,Oracle.W4DBN4.HDB.YTACCOUNT.REC.REMOTE_VERSION,Oracle.W4DBN4.HDB.YTACCOUNT.YTACCOUNT_BODY.<8,1>PLSQL_BLOCK.<13,5>ASSIGNMENT.40 REMOTE_VERSION,Oracle.W4DBN4.HDB.CUST_DOC_IN_STAT_AFTER.CUST_DOC_IN_STAT_AFTER_BODY.CALC_MTR.CALC_MTR_BODY.INIT_CARD_MTR.INIT_CARD_MTR_BODY.<193,5>PLSQL_BLOCK.<199,20>YGACCOUNT.REC.REMOTE_VERSION,Oracle.W4DBN4.HDB.ACCOUNT.REMOTE_VERSION,Oracle.W4DBN4.HDB.YTACCOUNT.YTACCOUNT_BODY.<8,1>PLSQL_BLOCK.<58,5>UPDATE.34 REMOTE_VERSION,Oracle.W4DBN4.HDB.YGACCOUNT.YGACCOUNT_BODY.<6,1>PLSQL_BLOCK.<10,5>SELECT INTO.35 REMOTE_VERSION', 'Oracle.W4DBN4.HDB.YGACCOUNT.YGACCOUNT_BODY.<6,1>PLSQL_BLOCK.<10,5>SELECT INTO.35 REMOTE_VERSION', 'PLSQL SelectInto', 'Oracle', 'W4DBN4', 'HDB', datetime.datetime(2025, 2, 6, 10, 42, 9)), ('ACCOUNT', 'Oracle.W4DBN4.HDB.YTACCOUNT.YTACCOUNT_BODY.<8,1>PLSQL_BLOCK.<12,14>YTACCOUNT.REC.REMOTE_

In [10]:
# db_connection.drop_table(connection = db_connect, table_name = LINEAGE_MANTA_TABLE_FLOW_TEST)

In [11]:



# table = extract_components(graph,'Raw_Node')
# # print(table)